## Crear DataFrames

Aquí aplicaremos la creación de DataFrames que se vió en el anterior Notebook, a modo de ejemplo para ver el uso y funcionamiento de diferentes funciones.

In [1]:
// Create the case classes for our domain
case class Department(id: String, name: String)
case class Employee(firstName: String, lastName: String, email: String, salary: Int)
case class DepartmentWithEmployees(department: Department, employees: Seq[Employee])

// Create the Departments
val department1 = new Department("123456", "Computer Science")
val department2 = new Department("789012", "Mechanical Engineering")
val department3 = new Department("345678", "Theater and Drama")
val department4 = new Department("901234", "Indoor Recreation")

// Create the Employees
val employee1 = new Employee("michael", "armbrust", "no-reply@berkeley.edu", 100000)
val employee2 = new Employee("xiangrui", "meng", "no-reply@stanford.edu", 120000)
val employee3 = new Employee("matei", null, "no-reply@waterloo.edu", 140000)
val employee4 = new Employee(null, "wendell", "no-reply@princeton.edu", 160000)
val employee5 = new Employee("michael", "jackson", "no-reply@neverla.nd", 80000)

// Create the DepartmentWithEmployees instances from Departments and Employees
val departmentWithEmployees1 = new DepartmentWithEmployees(department1, Seq(employee1, employee2))
val departmentWithEmployees2 = new DepartmentWithEmployees(department2, Seq(employee3, employee4))
val departmentWithEmployees3 = new DepartmentWithEmployees(department3, Seq(employee5, employee4))
val departmentWithEmployees4 = new DepartmentWithEmployees(department4, Seq(employee2, employee3))

Intitializing Scala interpreter ...

Spark Web UI available at http://ALC-1NJW5D3.usersad.everis.int:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1656667631263)
SparkSession available as 'spark'


defined class Department
defined class Employee
defined class DepartmentWithEmployees
department1: Department = Department(123456,Computer Science)
department2: Department = Department(789012,Mechanical Engineering)
department3: Department = Department(345678,Theater and Drama)
department4: Department = Department(901234,Indoor Recreation)
employee1: Employee = Employee(michael,armbrust,no-reply@berkeley.edu,100000)
employee2: Employee = Employee(xiangrui,meng,no-reply@stanford.edu,120000)
employee3: Employee = Employee(matei,null,no-reply@waterloo.edu,140000)
employee4: Employee = Employee(null,wendell,no-reply@princeton.edu,160000)
employee5: Employee = Employee(michael,jackson,no-reply@neverla.nd,80000)
departmentWithEmployees1: DepartmentWithEmployees = DepartmentWithEmp...


22/07/01 11:27:22 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


### Crear DataFrames a partir de una lista de las clases de casos

In [3]:
val departmentsWithEmployeesSeq1 = Seq(departmentWithEmployees1, departmentWithEmployees2)
val df1 = departmentsWithEmployeesSeq1.toDF()
df1.show()

val departmentsWithEmployeesSeq2 = Seq(departmentWithEmployees3, departmentWithEmployees4)
val df2 = departmentsWithEmployeesSeq2.toDF()
df2.show()

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|{123456, Computer...|[{michael, armbru...|
|{789012, Mechanic...|[{matei, null, no...|
+--------------------+--------------------+

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|{345678, Theater ...|[{michael, jackso...|
|{901234, Indoor R...|[{xiangrui, meng,...|
+--------------------+--------------------+



departmentsWithEmployeesSeq1: Seq[DepartmentWithEmployees] = List(DepartmentWithEmployees(Department(123456,Computer Science),List(Employee(michael,armbrust,no-reply@berkeley.edu,100000), Employee(xiangrui,meng,no-reply@stanford.edu,120000))), DepartmentWithEmployees(Department(789012,Mechanical Engineering),List(Employee(matei,null,no-reply@waterloo.edu,140000), Employee(null,wendell,no-reply@princeton.edu,160000))))
df1: org.apache.spark.sql.DataFrame = [department: struct<id: string, name: string>, employees: array<struct<firstName:string,lastName:string,email:string,salary:int>>]
departmentsWithEmployeesSeq2: Seq[DepartmentWithEmployees] = List(DepartmentWithEmployees(Department(345678,Theater and Drama),List(Employee(michael,jackson,no-reply@neverla.nd,80000), Employee(null,wende...


# Trabajar con DataFrames
## Unión de dos DataFrames

En esta sección veremos como unir dos dataframes para trabajar con ellos como si de uno solo se tratara.

In [4]:
val unionDF = df1.union(df2)
unionDF.show()

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|{123456, Computer...|[{michael, armbru...|
|{789012, Mechanic...|[{matei, null, no...|
|{345678, Theater ...|[{michael, jackso...|
|{901234, Indoor R...|[{xiangrui, meng,...|
+--------------------+--------------------+



unionDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [department: struct<id: string, name: string>, employees: array<struct<firstName:string,lastName:string,email:string,salary:int>>]


# Group by
Al igual que la cláusula "GROUP BY" de SQL, la función groupBy() de Spark se utiliza para reunir los datos idénticos en grupos en DataFrame/Dataset y realizar funciones de agregación en los datos agrupados. En este artículo, explicaré varios ejemplos de ```groupBy()``` con el lenguaje Scala.

El mismo enfoque se puede utilizar con Pyspark (Spark con Python).

Sintaxis:
```groupBy(col1 : scala.Predef.String, cols : scala.Predef.String*) :
      org.apache.spark.sql.RelationalGroupedDataset```

Cuando realizamos ```groupBy()``` en Spark Dataframe, devuelve el objeto RelationalGroupedDataset que contiene las siguientes funciones de agregación.

```count()``` - Devuelve el recuento de filas de cada grupo.

```mean()``` - Devuelve la media de los valores de cada grupo.

```max()``` - Devuelve el máximo de los valores de cada grupo.

```min()``` - Devuelve el mínimo de los valores de cada grupo.

```sum()``` - Devuelve el total de los valores de cada grupo.

```avg()``` - Devuelve la media de los valores de cada grupo.

```agg()``` - Utilizando la función agg(), podemos calcular más de un agregado a la vez.

```pivot()``` - Esta función se utiliza para pivotar el DataFrame que se verá más en profundidad más adelante.

## Preparando los datos y el DataFrame
Antes de empezar, vamos a crear el DataFrame a partir de una secuencia de los datos con los que vamos a trabajar. Este DataFrame contiene las columnas "employee_name", "department", "state", "salary", "age" y "bonus".

Utilizaremos este DataFrame de Spark para ejecutar ```groupBy()``` en las columnas "department" y calcular los agregados como mínimo, máximo, promedio, salario total para cada grupo utilizando las funciones de agregación ```min()```, ```max()``` y ```sum()``` respectivamente. y finalmente, también veremos cómo hacer grupos y agregados en múltiples columnas.

In [1]:
import spark.implicits._
val simpleData = Seq(("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  )
val df = simpleData.toDF("employee_name","department","state","salary","age","bonus")
df.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://ALC-1NJW5D3.usersad.everis.int:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1656916598302)
SparkSession available as 'spark'


22/07/04 08:36:55 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped
+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



import spark.implicits._
simpleData: Seq[(String, String, String, Int, Int, Int)] = List((James,Sales,NY,90000,34,10000), (Michael,Sales,NY,86000,56,20000), (Robert,Sales,CA,81000,30,23000), (Maria,Finance,CA,90000,24,23000), (Raman,Finance,CA,99000,40,24000), (Scott,Finance,NY,83000,36,19000), (Jen,Finance,NY,79000,53,15000), (Jeff,Marketing,CA,80000,25,18000), (Kumar,Marketing,NY,91000,50,21000))
df: org.apache.spark.sql.DataFrame = [employee_name: string, department: string ... 4 more fields]


## groupBy y agregados en las columnas del DataFrame
Hagamos el ```groupBy()``` en la columna 'department' del DataFrame y luego encontremos la suma del salario de cada departamento usando la función de agregada ```sum()```.

In [2]:
df.groupBy("department").sum("salary").show(false)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



Del mismo modo, podemos calcular el número de empleados de cada departamento mediante ```count()```.

In [3]:
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



Calcular el salario mínimo de cada departamento mediante ```min()```.

In [4]:
df.groupBy("department").min("salary").show()

+----------+-----------+
|department|min(salary)|
+----------+-----------+
|     Sales|      81000|
|   Finance|      79000|
| Marketing|      80000|
+----------+-----------+



Calcular el salario máximo de cada departamento mediante ```max()```.

In [5]:
df.groupBy("department").max("salary").show()

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|     Sales|      90000|
|   Finance|      99000|
| Marketing|      91000|
+----------+-----------+



Calcule el salario medio de cada departamento utilizando ```avg()```.

In [6]:
df.groupBy("department").avg("salary").show()

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+



Calcule el salario medio de cada departamento utilizando ```mean()```.

In [8]:
df.groupBy("department").mean("salary").show()

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+



## groupBy y aggregate en múltiples columnas del DataFrame
Del mismo modo, también podemos ejecutar ```groupBy()``` y ```aggregate()``` en dos o más columnas del DataFrame, el siguiente ejemplo hace group by en el 'department', 'state' y hace ```sum()``` en las columnas de 'salary' y 'bonus'.

In [9]:
//GroupBy en multiples columnas
df.groupBy("department","state")
    .sum("salary","bonus")
    .show(false)

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|Sales     |NY   |176000     |30000     |
|Sales     |CA   |81000      |23000     |
|Finance   |CA   |189000     |47000     |
|Finance   |NY   |162000     |34000     |
|Marketing |NY   |91000      |21000     |
|Marketing |CA   |80000      |18000     |
+----------+-----+-----------+----------+



Del mismo modo, podemos ejecutar ```gruopby()``` y ```aggregate()``` en dos o más columnas para otras funciones de agregación.

## Ejecutar más agregados a la vez
Usando la función de agregación ```agg()``` podemos calcular muchas agregaciones a la vez en una sola sentencia usando las funciones de agregación de Spark SQL ```sum()```, ```avg()```, ```min()```, ```max()``` ```mean()``` etc... Para usarlas, debemos importar ```import org.apache.spark.sql.functions._```

In [10]:
import org.apache.spark.sql.functions._

df.groupBy("department")
    .agg(
      sum("salary").as("sum_salary"),
      avg("salary").as("avg_salary"),
      sum("bonus").as("sum_bonus"),
      max("bonus").as("max_bonus"))
    .show(false)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
|Marketing |171000    |85500.0          |39000    |21000    |
+----------+----------+-----------------+---------+---------+



import org.apache.spark.sql.functions._


## Uso del filtro en los datos agregados
De forma similar a la cláusula "HAVING" de SQL, en Spark DataFrame podemos utilizar la función ```where()``` o ```filter()``` para filtrar las filas de los datos agregados.

In [11]:
df.groupBy("department")
    .agg(
      sum("salary").as("sum_salary"),
      avg("salary").as("avg_salary"),
      sum("bonus").as("sum_bonus"),
      max("bonus").as("max_bonus"))
    .where(col("sum_bonus") >= 50000)
    .show(false)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
+----------+----------+-----------------+---------+---------+

